# Deploy Llama3-8b on Amazon SageMaker using the LMI container.

[LMI containers](https://docs.djl.ai/docs/serving/serving/docs/lmi/index.html#overview-large-model-inference-lmi-containers) are a set of high-performance Docker Containers purpose built for large language model (LLM) inference. With these containers, you can leverage high performance open-source inference libraries like vLLM, TensorRT-LLM, Transformers NeuronX to deploy LLMs on AWS SageMaker Endpoints. 

These containers bundle together a model server with open-source inference libraries to deliver an all-in-one LLM serving solution.

In this notebook, we deploy the [llama3-8B](https://huggingface.co/meta-llama/Meta-Llama-3-8B) model on SageMaker by leveraging the [SageMaker Large Model Inference Container](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers).

# License agreement
View license information https://huggingface.co/meta-llama before using the model.
This notebook is a sample notebook and not intended for production use. Please refer to the licence at https://github.com/aws/mit-0.

## Install, import the required libraries, and set some variables

In [ ]:
%pip install sagemaker boto3 huggingface_hub awscli --upgrade  --quiet

In [ ]:
import json
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

## Select the container
SageMaker offers optimized [large model inference containers](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers) that contains different frameworks for model parallelism enabling inference of LLMs on multiple accelerators.

In this scenario we leverage the `NeuronX` container since we are deploying the model on `trn1`.

In [ ]:
neuron_image_uri = image_uris.retrieve(
    framework="djl-neuronx", region=sess.boto_session.region_name, version="0.28.0"
)

## Select the appropriate configuration parameters

To optimize the deployment of Large Language Models (LLMs); one needs to choose the appropriate model partitioning framework, optimal batching technique, batching size, tensor parallelism degree, etc. The choice of a particular configuration depends on the use case.
#### Set the configuration parameters using environment variables

1. `SERVING_LOAD_MODELS` - specifies the engine that will be used for this workload. In this case we'll be hosting a model using the **Python** engine.

2. `OPTION_MODEL_ID`: If you want to download the model from huggingface.co, you can set `OPTION_MODEL_ID` to the model id of a pre-trained model hosted inside a model repository on huggingface.co (https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co.
- If the model weights are in the standard HuggingFace format, runtime compilation is performed
- You can also provide compiled models and split-models. The source of the weights can be huggingface model hub, S3 or a local path.
If this is set to the URI of the Amazon S3 bucket that contains the model. When this is set, the container leverages [s5cmd](https://github.com/peak/s5cmd) to download the model from s3. This enables faster deployments by utilizing optimized approach within the DJL inference container to transfer the model from S3 into the hosting instance.

3. `OPTION_TENSOR_PARALLEL_DEGREE`: Set to the number of neuron cores over which the model needs to be partitioned. This parameter also controls the number of workers per model which will be started up when DJL serving runs. In this example we use the `ml.trn1.32xlarge` instance that has 16 accelerators with 2 neuron cores each, hence it is set to 32. More details about the [inferentia architecture](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/arch/neuron-hardware/trn1-arch.html).

4. `OPTION_ROLLING_BATCH`: This parameter enables the use of a particular batching technique for continuous or iteration level batching to enable merging multiple concurrent requests that arrive at different times for inference. In this scenario we set it to `auto` and the framework uses a custom rolling batch mechanism.

5. `OPTION_MAX_ROLLING_BATCH_SIZE`: The maximum number of concurrent requests to be used in a batch by the model server for inference. Clients can still send more requests to the endpoint, they will be queued.

6. `OPTION_N_POSITIONS`: This parameter controls the **Total sequence length, i.e. input sequence length + output sequence length** that you want the model to process during inference. If you are providing weights in standard huggingface format and performing runtime compilation, set this to the sequence length you would observe when using the endpoint.

For more information on the available options, please refer to the [DJL Serving - SageMaker Large Model Inference Configurations](https://github.com/deepjavalibrary/djl-serving/tree/master/serving/docs/lmi) and [NeuronX Configurations](https://github.com/deepjavalibrary/djl-serving/blob/master/serving/docs/lmi/user_guides/tnx_user_guide.md)


In [ ]:
env = {
    "HUGGINGFACE_HUB_CACHE": "/tmp",
    "TRANSFORMERS_CACHE": "/tmp",
    "SERVING_LOAD_MODELS": "test::Python=/opt/ml/model",
    "OPTION_MODEL_ID": "meta-llama/Meta-Llama-3-8B",
    "OPTION_TENSOR_PARALLEL_DEGREE": "2",  # inf2.8xlarge has 1 accelerator (with two NeuronCore-v2 cores).
    "OPTION_ROLLING_BATCH": "auto",
    "OPTION_MAX_ROLLING_BATCH_SIZE": "4",
    "OPTION_N_POSITIONS": "2048",
    "HF_TOKEN": <YOUR_HF_TOKEN>
}

In [ ]:
print(f"Environment variables are ---- > {env}")
print(f"Image going to be used is ---- > {neuron_image_uri}")

## To create the endpoint the steps are:
- Create the Model using the inference image container

- Create the endpoint config using the following key parameters

In this notebook we leverage the boto3 SDK. You can also use the [SageMaker SDK](https://sagemaker.readthedocs.io/en/stable/).

In [ ]:
instance_type = "ml.inf2.8xlarge"
endpoint_name = sagemaker.utils.name_from_base("Meta-Llama-3-8B")

## Create the Model
Leverage the `inference_image_uri` to create a model object.

In [ ]:
model_name = sagemaker.utils.name_from_base("lmi-Meta-Llama-3-8B")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": neuron_image_uri,
        "Environment": env,
    },
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

## Create an endpoint config
Create an endpoint configuration using the appropriate instance type. Set the `ContainerStartupHealthCheckTimeoutInSeconds` to account for the time taken to download the LLM weights from S3 or the model hub; and the time taken to load the model on the accelerators.

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600
        },
    ],
)
endpoint_config_response

## Create an endpoint using the model and endpoint config

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

#### This step can take ~15 mins or longer

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### Invoke the endpoint with a sample prompt

In [ ]:
# use this for Chatbot or QA or open ended generation task
prompt = "The meaning of life is "
params = {"max_new_tokens": 128, "do_sample": False}

In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

## Clean up the environment

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

#### Additional Resources:
- [Neuron Samples](https://github.com/aws-neuron/aws-neuron-samples)